# 🎤 IntervuAgent - LangGraph Interview Workflow

This notebook lets you **build, visualize, and test** the interview workflow step-by-step before integrating it into the app.

## 1. Setup & Imports

In [23]:
from typing import Annotated, Literal
from pydantic import BaseModel
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

## 2. Define State & Schemas

In [24]:
class InterviewState(BaseModel):
    messages: Annotated[list[BaseMessage], add_messages]

    student_name: str | None = None
    selected_topic: str | None = None

    stage: Literal[
        "ask_name",
        "extract_name",
        "ask_topic",
        "extract_topic",
        "ask_question",
        "await_answer",
        "end"
    ] = "ask_name"

    question_count: int = 0
    max_questions: int = 3

    intent: Literal["continue", "quit"] = "continue"
    should_end: bool = False

In [25]:
class NameExtraction(BaseModel):
    name: str

class TopicExtraction(BaseModel):
    topic: str

class EvalIntentOutput(BaseModel):
    feedback: str
    intent: Literal["continue", "quit"]

## 3. Initialize LLM

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()

# Main LLM for questions and evaluation
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.4,
    max_output_tokens=1024,
)

# Fast LLM for simple extractions (name, topic) - lower tokens = faster
llm_fast = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    max_output_tokens=256,
)

In [27]:
def route_by_stage(state: InterviewState):
    return state.stage

## 4. Define Graph Nodes

In [28]:
def ask_name_node(state: InterviewState):
    return {
        "messages": [
            AIMessage(content="Hey there! 👋 I'm your friendly AI interviewer. Before we start, what's your name?")
        ],
        "stage": "extract_name"
    }

def extract_name_node(state: InterviewState):
    structured_llm = llm_fast.with_structured_output(NameExtraction)
    last_user_msg = state.messages[-1].content
    result = structured_llm.invoke([
        SystemMessage(content="Extract only the first name from the user message. Return JSON."),
        HumanMessage(content=last_user_msg)
    ])
    return {
        "student_name": result.name,
        "stage": "ask_topic"
    }

def ask_topic_node(state: InterviewState):
    return {
        "messages": [
            AIMessage(content=f"Awesome, {state.student_name}! 🎯 What topic would you like to practice? "
                             f"(e.g. Python, JavaScript, SQL, React, Java, C++, etc.)")
        ],
        "stage": "extract_topic"
    }

def extract_topic_node(state: InterviewState):
    structured_llm = llm_fast.with_structured_output(TopicExtraction)
    last_user_msg = state.messages[-1].content
    result = structured_llm.invoke([
        SystemMessage(content="Extract only the technical topic name. Return JSON."),
        HumanMessage(content=last_user_msg)
    ])
    return {
        "selected_topic": result.topic,
        "stage": "ask_question"
    }

def ask_question_node(state: InterviewState):
    # Only send the last 4 messages for context (faster, saves tokens)
    recent_msgs = state.messages[-4:] if len(state.messages) > 4 else state.messages

    messages = [
        SystemMessage(content=f"""You are a friendly, encouraging technical interviewer talking to a BEGINNER.
The student's name is {state.student_name} and the topic is {state.selected_topic}.

Rules:
- Ask ONE simple, beginner-level question.
- Think 'first week of learning' level - basic concepts, definitions, simple use cases.
- Maximum 2 sentences.
- Do NOT ask tricky or advanced questions.
- Sound warm and human, like a supportive mentor.
- Do NOT repeat a question already asked.
- Do NOT explain the answer.""")
    ]
    messages.extend(recent_msgs)
    messages.append(
        HumanMessage(content=f"Ask a beginner-level question about {state.selected_topic}. Question #{state.question_count + 1}")
    )
    response = llm.invoke(messages)
    return {
        "messages": [AIMessage(content=response.content)],
        "stage": "await_answer"
    }

def evaluate_and_check_node(state: InterviewState):
    structured_llm = llm.with_structured_output(EvalIntentOutput)
    last_user_msg = state.messages[-1].content

    for attempt in range(3):
        try:
            result = structured_llm.invoke([
                SystemMessage(content="""You are a warm, encouraging interviewer giving feedback to a BEGINNER.

1. Give short, friendly feedback (2-3 sentences max). Be encouraging!
2. If the answer is wrong, gently correct them without being harsh.
3. If the student says anything like 'stop', 'quit', 'exit', 'end', 'done',
   'no more', 'that is enough', 'I want to stop', or similar, set intent to 'quit'.
4. Otherwise set intent to 'continue'.

Return valid JSON only. No markdown wrapping."""),
                HumanMessage(content=last_user_msg)
            ])
            break
        except Exception:
            if attempt == 2:
                result = EvalIntentOutput(feedback="Nice effort! Let's keep going.", intent="continue")
            continue

    new_count = state.question_count + 1
    should_continue = result.intent == "continue" and new_count < state.max_questions
    return {
        "messages": [AIMessage(content=result.feedback)],
        "intent": result.intent,
        "question_count": new_count,
        "stage": "ask_question" if should_continue else "end"
    }

In [29]:
def end_node(state: InterviewState):
    return {
        "messages": [AIMessage(content=f"Great job {state.student_name}! 🎉 That was a solid practice session on {state.selected_topic}. Keep learning and you'll do amazing. Good luck! 🚀")],
        "should_end": True
    }

## 5. Build the Graph

### ⚡ Key design: `interrupt_before` pauses for user input

The graph pauses **before** `extract_name`, `extract_topic`, and `evaluate_answer` nodes.  
Your code collects user input, injects it via `graph.update_state()`, and resumes with `graph.invoke(None)`.  
This avoids redundant LLM calls — the graph only runs when it has real input to process.

In [30]:
workflow = StateGraph(InterviewState)

workflow.add_node("ask_name", ask_name_node)
workflow.add_node("extract_name", extract_name_node)
workflow.add_node("ask_topic", ask_topic_node)
workflow.add_node("extract_topic", extract_topic_node)
workflow.add_node("ask_question", ask_question_node)
workflow.add_node("evaluate_answer", evaluate_and_check_node)
workflow.add_node("end", end_node)

workflow.set_entry_point("ask_name")

workflow.add_conditional_edges("ask_name", route_by_stage, {"extract_name": "extract_name"})
workflow.add_conditional_edges("extract_name", route_by_stage, {"ask_topic": "ask_topic"})
workflow.add_conditional_edges("ask_topic", route_by_stage, {"extract_topic": "extract_topic"})
workflow.add_conditional_edges("extract_topic", route_by_stage, {"ask_question": "ask_question"})
workflow.add_conditional_edges("ask_question", route_by_stage, {"await_answer": "evaluate_answer"})
workflow.add_conditional_edges("evaluate_answer", route_by_stage, {
    "ask_question": "ask_question",
    "end": "end"
})

workflow.add_edge("end", END)

memory = MemorySaver()

graph = workflow.compile(
    checkpointer=memory,
    interrupt_before=["extract_name", "extract_topic", "evaluate_answer"]
)

## 6. Visualize the Graph

In [31]:
import IPython.display as display

mermaid_code = graph.get_graph().draw_mermaid()
display.Markdown(f"```mermaid\n{mermaid_code}\n```")

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	ask_name(ask_name)
	extract_name(extract_name<hr/><small><em>__interrupt = before</em></small>)
	ask_topic(ask_topic)
	extract_topic(extract_topic<hr/><small><em>__interrupt = before</em></small>)
	ask_question(ask_question)
	evaluate_answer(evaluate_answer<hr/><small><em>__interrupt = before</em></small>)
	end(end)
	__end__([<p>__end__</p>]):::last
	__start__ --> ask_name;
	ask_name -.-> extract_name;
	ask_question -. &nbsp;await_answer&nbsp; .-> evaluate_answer;
	ask_topic -.-> extract_topic;
	evaluate_answer -.-> ask_question;
	evaluate_answer -.-> end;
	extract_name -.-> ask_topic;
	extract_topic -.-> ask_question;
	end --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

## 7. Run the Interview

In [32]:
import uuid

# Auto-generate a unique session ID each run
config = {"configurable": {"thread_id": f"interview-{uuid.uuid4().hex[:8]}"}}

# STEP 1: Start the graph
graph.invoke({"messages": []}, config=config)

# Track how many messages we've already displayed
state = graph.get_state(config)
all_msgs = state.values.get("messages", [])
displayed_count = 0

# Print AI greeting
for msg in all_msgs:
    if isinstance(msg, AIMessage):
        print(f"\n🤖 AI: {msg.content}\n")
displayed_count = len(all_msgs)

# STEP 2: Main loop
while True:
    # Check if graph has finished (no more nodes to run)
    if not graph.get_state(config).next:
        print("\n✅ Interview complete!")
        break

    user_input = input("👤 You: ").strip()
    if not user_input:
        continue

    # Manual quit keywords (instant exit, no LLM call)
    if user_input.lower() in ("quit", "exit", "stop", "end", "bye", "done"):
        print("\n👋 Interview ended. Thanks for practicing!")
        break

    print(f"\n👤 You: {user_input}\n")
    print("⏳ Thinking...\n")

    # Resume graph with user input
    graph.update_state(config, {"messages": [HumanMessage(content=user_input)]})
    graph.invoke(None, config=config)

    # Print ALL new AI messages (feedback + next question)
    state = graph.get_state(config)
    all_msgs = state.values.get("messages", [])
    new_msgs = all_msgs[displayed_count:]

    for msg in new_msgs:
        if isinstance(msg, AIMessage):
            print(f"🤖 AI: {msg.content}\n")

    displayed_count = len(all_msgs)

    # Check if interview ended naturally (max questions or user said quit in answer)
    if state.values.get("should_end", False):
        print("\n✅ Interview complete!")
        break


🤖 AI: Hey there! 👋 I'm your friendly AI interviewer. Before we start, what's your name?




👤 You: hi my name is jayanth

⏳ Thinking...

🤖 AI: Awesome, jayanth! 🎯 What topic would you like to practice? (e.g. Python, JavaScript, SQL, React, Java, C++, etc.)


👤 You: sql please

⏳ Thinking...

🤖 AI: Great choice, jayanth! SQL is super useful.

To kick things off, in your own words, what would you say SQL is primarily used for? No need for a perfect definition, just your understanding!


👤 You: sql is an language to create, manage the databse systems , it has DML, DQL,TCL, DRL

⏳ Thinking...

🤖 AI: That's a great start! You're absolutely right that SQL is used for creating and managing databases. You've correctly identified DML, DQL, and TCL. We also have DDL for defining the database structure and DCL for controlling access, and DRL is often grouped under DQL.

🤖 AI: Excellent, jayanth! You've got a good handle on the different categories of SQL commands.

Let's move on to one of the most common commands. What do you think the `SELECT` statement is used for in SQL?


👤 You: SE

## 8. Debug: Inspect Messages

In [33]:
for msg in state.values.get("messages", []):
    print(type(msg))
    print(msg.model_dump())
    print("----")

<class 'langchain_core.messages.ai.AIMessage'>
{'content': "Hey there! 👋 I'm your friendly AI interviewer. Before we start, what's your name?", 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': '49639bd7-f2bc-452f-8f9f-fa5fd66de351', 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': None}
----
<class 'langchain_core.messages.human.HumanMessage'>
{'content': 'hi my name is jayanth', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '857fe669-21f7-4de4-9ac6-6fbe4b1f4c11'}
----
<class 'langchain_core.messages.ai.AIMessage'>
{'content': 'Awesome, jayanth! 🎯 What topic would you like to practice? (e.g. Python, JavaScript, SQL, React, Java, C++, etc.)', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': '73045422-96f4-46c2-881f-231b55fda5ef', 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': None}
----
<class 'langchain_core.messages.human.HumanMessage'>
{'content':